In [ ]:
import pickle
import pandas as pd
import numpy as np
import cv2
import os
import cuml
from matplotlib import pyplot as plt

opj = os.path.join

In [ ]:
with open('../input/bestfitting-ml/feats.pickle', 'rb') as handle:
    feats = pickle.load(handle)

In [ ]:
with open('../input/bestfitting-ml-public/feats_ext.pickle', 'rb') as handle:
    feats_ext = pickle.load(handle)

In [ ]:
feats.shape

In [ ]:
feats_ext.shape

In [ ]:
df = pd.read_csv('../input/bestfitting-ml/feats_df.csv')
df['dataset'] = 'train'
df.head()

In [ ]:
df_ext = pd.read_csv('../input/bestfitting-ml-public/feats_df_ext.csv')
df_ext['dataset'] = 'public'
df_ext.head()

In [ ]:
all_feats = np.vstack([feats, feats_ext])
all_feats.shape

In [ ]:
def read_rgby(img_id,train=True):
    if train: img_dir = '../input/hpa-single-cell-image-classification/train'
    else: img_dir = '../input/hpa-public-768-excl-0-16/hpa_public_excl_0_16_768/small'
    suffix = '.png'
    colors = ['red', 'green', 'blue'] #, 'yellow']
    flags = cv2.IMREAD_GRAYSCALE
    img = [cv2.imread(opj(img_dir, img_id + '_' + color + suffix), flags)
           for color in colors]
    img = np.stack(img, axis=-1)
    return img

In [ ]:
df = pd.concat([df,df_ext], ignore_index=True)

In [ ]:
umap = cuml.UMAP(n_neighbors=2, n_components=5)

In [ ]:
ump = umap.fit_transform(all_feats)

In [ ]:
ump.shape

In [ ]:
all_feats.shape

In [ ]:
# Compute DBSCAN
db = cuml.cluster.DBSCAN(eps=0.05, min_samples=2).fit(ump)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
clusters = set(list(labels))

d = {}
for c in clusters:
    d[c] = 0
for l in labels:
    d[l] += 1
d

In [ ]:
f = [i for i,x in enumerate(list(labels)) if x == 5505]

for k in f:
    
    train = True
    if k > len(feats): train=False
    
    img_id = df.ID.loc[k]
    img_lbl = df.Label.loc[k]
    
    plt.figure(figsize=(10,10))
    
    plt.subplot(1,1,1)
    img = read_rgby(img_id, train)
    plt.imshow(img)
    plt.title(f'Image: {img_id}, label: {img_lbl}')
    
    plt.show()

In [ ]:
labels.shape

In [ ]:
len(df)

In [ ]:
df['cluster'] = labels

In [ ]:
df = df[['ID', 'Label', 'cluster', 'dataset']]

In [ ]:
df

In [ ]:
df.to_csv('combined.csv', index=False)

In [ ]:
f = [48868, 48869]
for k in f:
    
    train = True
    if k > len(feats): train=False
    
    img_id = df.ID.loc[k]
    img_lbl = df.Label.loc[k]
    
    plt.figure(figsize=(10,10))
    
    plt.subplot(1,1,1)
    img = read_rgby(img_id, train)
    plt.imshow(img)
    plt.title(f'Image: {img_id}, label: {img_lbl}')
    
    plt.show()